In [54]:
import requests

In [55]:
standings_url = "https://fbref.com/en/comps/9/2021-2022/2021-2022-Premier-League-Stats"

In [56]:
data = requests.get(standings_url)

In [57]:
from bs4 import BeautifulSoup

In [58]:
soup = BeautifulSoup(data.text)

In [59]:
#Select the table we are gonna use via CSS selector

standings_table = soup.select('table.stats_table')[0]

In [60]:
#First step for getting the link to a specific team's data, retrieving html elements with the table with an <a> tag

links = standings_table.find_all('a')

In [61]:
#Get the href within the a tags

links = [l.get('href') for l in links]

In [62]:
#Actually get the partial link, we filter to only include links that contain /squads/

links = [l for l in links if '/squads/' in l]

In [63]:
#Include the base url so we can get a full link

team_urls = [f"https://fbref.com{l}" for l in links]

In [64]:
#We are going to get the data for the first team in the team_urls list

team_url = team_urls[0]

In [65]:
data = requests.get(team_url)

In [66]:
import pandas as pd

In [ ]:
#Use pandas to read the html for the table on the website named "Scores & Fixtures" and save it in a dataframe

matches = pd.read_html(data.text, match ="Scores & Fixtures")

/var/folders/5k/6dck_vks5csd7thvlkxq9jl80000gn/T/ipykernel_84117/498878196.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match ="Scores & Fixtures")


In [68]:
soup = BeautifulSoup(data.text)

In [69]:
#Similar to what we did previously, wer are getting the links for the Shooting stats for each team

links = soup.find_all('a')

In [70]:
links = [l.get('href') for l in links]

In [71]:
links = [l for l in links if l and 'all_comps/shooting' in l]

In [72]:
data = requests.get(f"https://fbref.com{links[0]}")

In [ ]:
#Read the HTMl and asve data using pandas

shooting = pd.read_html(data.text, match='Shooting')[0]

In [50]:
shooting.columns = shooting.columns.droplevel()

In [ ]:
#Since we have repeated data in the "matches" and "shooting" dataframes, we can merge them together to get the data we want

team_data = matches[0].merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [53]:
#Now we are going to do the same thing, but for multiple teams and seasons

years = list(range(2022, 2020, -1))

In [80]:
all_matches = []

In [85]:
standings_url = "https://fbref.com/en/comps/9/2021-2022/2021-2022-Premier-League-Stats"

In [95]:
#Building the loop to do basically everything we did before but for multiple years and teams
import time

for year in years:
    #This first part of the loop retrieves all teams for a given year
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text, 'html.parser')
    standings_table = soup.select('table.stats_table')[0]

    #Building a link to the data for each team
    links = [l.get('href') for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]

    previous_season = soup.select('a.prev')[0].get('href')
    standings_url = f"https://fbref.com{previous_season}"

    #This next part of the loop retrieves the data for each team
    for team in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")

        #Dataframe with Scores and Fixtures data
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

        #Dataframe with Shooting data
        soup = BeautifulSoup(data.text, 'html.parser')
        links = [l.get('href') for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()

        #Merging the two dataframes into one with all data. If a team does not have shooting data, the code will skip it
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue

        team_data = team_data[team_data["Comp"] == "Premier League"]
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1.5)

IndexError: list index out of range

In [ ]:
#all_matches is a list of dataframes, so we concatenate them

match_df = pd.concat(all_matches)

In [ ]:
match_df.columns = [c.lower() for c in match_df.columns]

In [ ]:
match_df.to_csv('scrapedData.csv')